In [14]:
!pip install langchain openai
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [15]:
# imports
from langchain.chat_models import ChatOpenAI
import getpass
import os

In [1]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Modelos

In [16]:
llm = ChatOpenAI(model="gpt-4o-mini")

<ipython-input-16-fe268a76a9ba>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini")


In [17]:
response = llm.invoke("Olá")

print(response)

content='Olá! Como posso ajudar você hoje?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None} id='run-fdca256a-aab6-4bc9-a732-e45c1d1258b3-0'


## Prompts

### Templates Simples

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: Artificial Intelligence is the future!', additional_kwargs={}, response_metadata={})]


In [19]:
response = llm.invoke(prompt)

print(response.content)

Inteligência Artificial é o futuro!


### Templates de Mensagens

In [20]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

# messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

content='Olá, como você está?' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 36, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None} id='run-721ae067-9182-4c8d-b246-0ec6c79110cd-0'


In [21]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [22]:
prompt = prompt_template.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "português",
    "texto": "Hello, how are you?"
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]


In [23]:
response = llm.invoke(prompt)

print(response.content)

Olá, como você está?


### Parsers

In [24]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [25]:
output = parser.invoke(response)

print(output)

Olá, como você está?


## Encadeamento

In [26]:
chain = prompt_template | llm | parser

In [27]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [28]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [29]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

¡Las playas de Recife tienen tiburones!


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [30]:
# Template do prompt
prompt_template = ChatPromptTemplate.from_template(
    "Crie uma piada relacionada ao seguinte tópico: {topico}"
)

# Parser para processar o resultado
parser = StrOutputParser()

# Encadeando os componentes
chain = prompt_template | llm | parser

# Função para gerar piada
def criar_piada(topico):
    response = chain.invoke({"topico": topico})
    return response

# Exemplo de uso
topico_usuario = "tecnologia"
piada = criar_piada(topico_usuario)
print("Piada gerada:", piada)

Piada gerada: Por que o computador foi ao médico?

Porque ele tinha um vírus! 🦠💻


### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [34]:
# Template do prompt
prompt_template = ChatPromptTemplate.from_template(
    "Analise o texto, classificando entre positivo, neutro ou negativo {texto}. Seja direto, respondendo a classificação e uma breve justificativa"
)

# Parser para processar o resultado
parser = StrOutputParser()

# Encadeando os componentes
chain = prompt_template | llm | parser

# Função para gerar piada
def criar_piada(texto):
    response = chain.invoke({"texto": texto})
    return response

# Exemplo de uso
texto_usuario = "Hoje acordei bastante animado"
texto = criar_piada(texto_usuario)
print("Resposta:", texto)

Resposta: Classificação: Positivo. 

Justificativa: O uso da palavra "animado" indica um estado de ânimo elevado e otimista, sugerindo uma sensação de felicidade ou entusiasmo.


### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [35]:
# Template do prompt
prompt_template = ChatPromptTemplate.from_template(
    "Escreva o código de uma função Python de acordo com a seguinte descrição: {descricao}"
)

# Parser para processar o resultado
parser = StrOutputParser()

# Encadeando os componentes
chain = prompt_template | llm | parser

# Função para gerar o código
def gerar_funcao(descricao):
    response = chain.invoke({"descricao": descricao})
    return response

# Exemplo de uso
descricao_usuario = "Uma função que soma dois números e retorna o resultado."
codigo_gerado = gerar_funcao(descricao_usuario)
print("Código gerado:")
print(codigo_gerado)

Código gerado:
Claro! Aqui está um exemplo de uma função em Python que soma dois números e retorna o resultado:

```python
def somar(a, b):
    """
    Esta função recebe dois números e retorna a soma deles.

    Args:
    a (int, float): O primeiro número.
    b (int, float): O segundo número.

    Returns:
    int, float: A soma dos dois números.
    """
    return a + b

# Exemplo de uso
resultado = somar(3, 5)
print(f"A soma é: {resultado}")
```

Nesta função, `somar` aceita dois parâmetros, `a` e `b`, que podem ser inteiros ou números de ponto flutuante. A função retorna a soma desses dois números. O exemplo de uso no final mostra como chamar a função e imprimir o resultado.


### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [37]:
# Configurando o modelo
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Template 1: Explicar o tópico de forma simplificada
explicacao_template = ChatPromptTemplate.from_template(
    "Explique de forma simples o seguinte tópico: {topico}. Seja direto"
)

# Template 2: Traduzir a explicação para inglês
traducao_template = ChatPromptTemplate.from_template(
    "Traduza o seguinte texto para inglês: {texto}. Seja direto"
)

# Parser para processar os resultados
parser = StrOutputParser()

# Encadeando os dois templates
chain_explicacao = explicacao_template | llm | parser
chain_traducao = traducao_template | llm | parser

# Função para realizar ambas as operações
def explicar_e_traduzir(topico):
    explicacao = chain_explicacao.invoke({"topico": topico})
    traducao = chain_traducao.invoke({"texto": explicacao})
    return explicacao, traducao

# Exemplo de uso
topico_usuario = "inteligência artificial"
explicacao, traducao = explicar_e_traduzir(topico_usuario)

print("Explicação em português:")
print(explicacao)
print("\nTradução para inglês:")
print(traducao)

Explicação em português:
Inteligência artificial é um ramo da ciência da computação que visa criar máquinas e sistemas que podem realizar tarefas que normalmente precisariam de intervenção humana. Isso inclui coisas como reconhecimento de voz, aprendizado, planejamento e resolução de problemas. A ideia é que essas máquinas possam "pensar" e "aprender" de forma semelhante aos humanos.

Tradução para inglês:
Artificial intelligence is a branch of computer science that aims to create machines and systems capable of performing tasks that typically require human intervention. This includes things like voice recognition, learning, planning, and problem-solving. The idea is that these machines can "think" and "learn" in a way similar to humans. Be direct.


### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [39]:
# Configurando o modelo
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Template do prompt
prompt_template = ChatPromptTemplate.from_template(
    "Você é um especialista na instituição CESAR School. Responda de forma clara e objetiva à seguinte pergunta: {pergunta}"
)

# Parser para processar o resultado
parser = StrOutputParser()

# Encadeando os componentes
chain = prompt_template | llm | parser

# Função para responder perguntas sobre CESAR School
def responder_pergunta(pergunta):
    response = chain.invoke({"pergunta": pergunta})
    return response

# Exemplo de uso
pergunta_usuario = "Quais são os cursos oferecidos pela CESAR School?"
resposta = responder_pergunta(pergunta_usuario)
print("Resposta:", resposta)

Resposta: A CESAR School oferece os seguintes cursos:

1. Graduação em Design
2. Graduação em Ciência da Computação
3. Mestrado Profissional em Engenharia de Software
4. Mestrado Profissional em Design de Interação
5. Pós-Graduação em Metodologias Ágeis
6. Pós-Graduação em Experiência do Usuário e Design de Interfaces.

Além desses, a CESAR School também oferece cursos de extensão e programas de residência para profissionais da área de tecnologia.
